In [ ]:
!mim download mmsegmentation --config pspnet_r50-d8_4xb2-40k_cityscapes-512x1024 --dest .

In [ ]:
# # test a video and show the results
# video = mmcv.VideoReader('video.mp4')
# for frame in video:
#    result = inference_model(model, frame)
#    show_result_pyplot(model, frame, result, wait_time=1)

In [1]:
from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv
from glob import glob 
import os 
import numpy as np 

In [4]:
import cv2

In [2]:
# init model
config_file = 'deeplabv3plus_r101_512x512_face-occlusion.py'
checkpoint_file = 'deeplabv3plus_r101_512x512_face-occlusion-93ec6695.pth'
model = init_model(config_file, checkpoint_file, device='cuda:0')

/local/home/yaqqin/miniconda3/envs/work38/lib/python3.8/site-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/local/home/yaqqin/miniconda3/envs/work38/lib/python3.8/site-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/local/home/yaqqin/miniconda3/envs/work38/lib/python3.8/site-packages/mmseg/models/losses/cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
/lo

Loads checkpoint by local backend from path: deeplabv3plus_r101_512x512_face-occlusion-93ec6695.pth


In [8]:
path = '../outputs/justin/00001.png'
img = mmcv.imread(path)
file_name = os.path.split(path)[-1]
result = inference_model(model, img)
show_result_pyplot(model, img, result, show=True, out_file=f"seg_output/{file_name}", opacity=0.5)

/local/home/yaqqin/miniconda3/envs/work38/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


array([[[20, 31, 45],
        [20, 31, 45],
        [20, 31, 45],
        ...,
        [57, 56, 73],
        [57, 56, 73],
        [57, 56, 73]],

       [[20, 31, 45],
        [20, 31, 45],
        [20, 31, 45],
        ...,
        [57, 56, 73],
        [57, 56, 73],
        [57, 56, 73]],

       [[20, 31, 45],
        [20, 31, 45],
        [20, 31, 45],
        ...,
        [57, 56, 73],
        [57, 56, 73],
        [57, 56, 73]],

       ...,

       [[ 8,  6,  8],
        [ 8,  6,  8],
        [ 8,  6,  8],
        ...,
        [14, 10, 14],
        [14, 10, 14],
        [14, 10, 14]],

       [[ 8,  6,  8],
        [ 8,  6,  8],
        [ 8,  6,  8],
        ...,
        [14, 10, 14],
        [14, 10, 14],
        [14, 10, 14]],

       [[ 8,  6,  8],
        [ 8,  6,  8],
        [ 8,  6,  8],
        ...,
        [14, 10, 14],
        [14, 10, 14],
        [14, 10, 14]]], dtype=uint8)

numpy.ndarray

In [5]:
result.keys()

['seg_logits', 'pred_sem_seg']

In [9]:
import numpy as np 
seg_mask = np.asanyarray(result.pred_sem_seg.values()[0].to('cpu'))
np.unique(seg_mask)

array([0, 1])

In [16]:
result.pred_sem_seg.values()

[tensor([[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]], device='cuda:0')]

In [10]:
seg_mask.shape

(1, 1080, 1920)